In [1]:
import numpy as np
from scipy.optimize import Bounds, NonlinearConstraint, minimize

import matplotlib.pyplot as plt
import qutip
from qutip import expect, fidelity, qutrit_basis, qutrit_ops

import sys
sys.path.append('../')
from common.common import tsoa_coupled_spins_quantum_simulation

* Define the control process as an objective function which should be minimized
* Input params will be the params of the trigonometric series
* we will use 3 harmonics, so we need 2*3 + 2 = 8 params

In [2]:
nb_harmonics = 5
nb_params = 2 * (2 * nb_harmonics + 1)
max_steps = 200
ξ = 1

* Parameter bounds and constraints of the system

In [3]:
upper_bound = 1 * np.ones(nb_params)
lower_bound = -1 * np.ones(nb_params)
bounds = Bounds(lower_bound, upper_bound)

def omega(t, params):
    rabi_freq_params = params[0:2 * nb_harmonics + 1]

    Ω = rabi_freq_params[0]

    for harmonic in range(1, nb_harmonics + 1):
        Ω += rabi_freq_params[2 * harmonic - 1] * np.cos(harmonic * t) + rabi_freq_params[2 * harmonic] * np.sin(harmonic * t)

    return Ω

def detuning(t, params):
    detuning_params = params[2 * nb_harmonics + 1: len(params)]

    Δ = detuning_params[0]

    for harmonic in range(1, nb_harmonics + 1):
        Δ += detuning_params[2 * harmonic - 1] * np.cos(harmonic * t) + detuning_params[2 * harmonic] * np.sin(harmonic * t)

    return Δ

* Scipy algorithm execution

In [ ]:
# (T, max_control)
optimizations = [(2.45, 1.), (1.4, 2.), (1.2, 3.), (1.1, 4.)]
optimal_params_array = []
omega_params_array = []
detuning_params_array = []
fidelities_array = []
omegas_array = []
detuning_array = []
timings_array = []
max_control_array = [1, 2, 3, 4]
states_array = []

for (T, max_control) in optimizations:
    initial_params = np.zeros(nb_params)

    final_time = T

    def const_fun1(params):
        time = np.linspace(start = 0, stop = final_time, num = max_steps)
        omegas = [omega(t, params = params) for t in time]

        return np.max(np.abs(omegas))

    def const_fun2(params):
        time = np.linspace(start = 0, stop = final_time, num = max_steps)
        detunings = [detuning(t, params = params) for t in time]

        return np.max(np.abs(detunings))

    constraints = [NonlinearConstraint(const_fun1, -max_control, max_control), NonlinearConstraint(const_fun2, -max_control, max_control)]

    def tsoa_objective_function(input_params):
        basis3lvl = qutrit_basis()
        initial_quantum_state = basis3lvl[0]
        target_quantum_state = basis3lvl[1]

        states, _, _ = tsoa_coupled_spins_quantum_simulation(input_params=input_params, 
                                    initial_quantum_state=initial_quantum_state,
                                    final_time=T, nb_harmonics=nb_harmonics, max_steps=max_steps, ξ=ξ)

        created_quantum_state = states[-1]
        
        actual_fidelity = fidelity(target_quantum_state, created_quantum_state) ** 2
        infidelity = 1 - actual_fidelity

        return infidelity
    
    def gradient(params):
        nb_params = len(params)
        perturbation = 1e-3

        grad_vector = []

        for param in range(nb_params):
            # create the perturbation array for the params
            epsilon = np.zeros(nb_params)
            epsilon[param] += perturbation

            diff = (tsoa_objective_function(params + epsilon) - tsoa_objective_function(params)) / perturbation

            grad_vector.append(diff)

        return np.array(grad_vector)

    res = minimize(tsoa_objective_function, initial_params, method='SLSQP', jac=gradient,
               options={'ftol': 1e-10, 'disp': True},
               constraints=constraints)
    
    optimal_params = res.x
    optimal_params_array.append(optimal_params)

    omega_params = optimal_params[0:2 * nb_harmonics + 1]
    detuning_params = optimal_params[2 * nb_harmonics + 1: len(optimal_params)]

    omega_params_array.append(optimal_params)
    detuning_params_array.append(detuning_params)

    time = np.linspace(start = 0, stop = final_time, num = max_steps)
    
    omegas_array.append(omega(time, optimal_params))
    detuning_array.append(detuning(time, optimal_params))
    timings_array.append(time)
    
    initial_quantum_state = qutrit_basis()[0]
    target_quantum_state = qutrit_basis()[1]

    states, omegas, detunings = tsoa_coupled_spins_quantum_simulation(input_params=optimal_params, initial_quantum_state=initial_quantum_state,
                                                final_time=T, nb_harmonics=nb_harmonics,max_steps=max_steps, ξ=ξ)
    
    states_array.append(states)

    desired_state = qutrit_basis()[1]
    
    fidelities = [fidelity(desired_state, state) ** 2 for state in states]
    fidelities_array.append(fidelities)

* Print quantum system evolution with the best solution for the trigonometric series parameters

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])
fig.set_figheight(8)
fig.set_figwidth(12)

axes = [ax1, ax2, ax3, ax4]
subplot_params = [(.16, .75, .1, .1), (.60, .57, .1, .1), (.17, .15, .1, .1), (.60, .15, .1, .1)]
titles = ['a', 'b', 'c', 'd']

for i, ax in enumerate(axes):
    omegas = omegas_array[i]
    detunings = detuning_array[i]
    time = timings_array[i]
    max_control = max_control_array[i]
    
    ax.plot(time, omegas, label = 'Ω')
    ax.plot(time, detunings, label = 'Δ')
    ax.set_ylabel(r"Ω,Δ ($\xi$)", rotation = 90)
    ax.set_xlabel(r"t ($\xi^{-1}$)")
    ax.set_ylim((-(max_control + 0.1), (max_control + 0.1)))
    ax.set_title('(' + titles[i] + ')', loc = "right", fontsize = 10)
    ax.legend(loc='center right')

    l, b, h, w = subplot_params[i]
    fidelities = fidelities_array[i]
    ax5 = fig.add_axes([l, b, w, h])
    ax5.plot(time, fidelities)
    ax5.set_ylabel("Fidelity", rotation = 90)

In [ ]:
np.round([(states_array[i][-1].full()[0], states_array[i][-1].full()[1] / np.sqrt(2), states_array[i][-1].full()[1] / np.sqrt(2), states_array[i][-1].full()[2]) for i in range(4)], 4)

In [ ]:
np.round(np.abs([(states_array[i][-1].full()[0], states_array[i][-1].full()[1] / np.sqrt(2), states_array[i][-1].full()[1] / np.sqrt(2), states_array[i][-1].full()[2]) for i in range(4)])**2, 4)

In [ ]:
optimal_params_array[0]

In [ ]:
fidelities = fidelities_array[0]
max_concurrence = round(np.max(fidelities), 4)

omegas = omegas_array[0]
detunings = detuning_array[0]
time = timings_array[0]
max_control = max_control_array[0]
states = states_array[0]

[proj1, proj2, proj3, trans12, trans23, _] = qutrit_ops()
# time_span = np.linspace(start = 0, stop = T, num = max_steps)
time_span = time

population1 = expect(proj1.dag() * proj1, states)
population2 = expect(proj2.dag() * proj2, states)
population3 = expect(proj3.dag() * proj3, states)

# fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
fig = plt.figure()
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax4 = fig.add_subplot(gs[1, :-1])
fig.set_figheight(8)
fig.set_figwidth(12)
# fig.suptitle(f"Trigonometric Series - {nb_harmonics} harmonics - Concurrence {max_concurrence}")

ax1.plot(time_span, omegas, label = 'Ω')
ax1.plot(time_span, detunings, label = 'Δ')
ax1.set_ylabel(r"Ω,Δ ($\xi$)", rotation = 90)
ax1.set_xlabel(r"t ($\xi^{-1}$)")
ax1.set_ylim((-1.1, 1.1))
ax1.legend()
ax1.set_title('(a)', loc = "right", fontsize = 10)

ax2.plot(time_span, fidelities)
ax2.axhline(y = 0.9999, color = 'r', linestyle = '--', label = '0.9999')
ax2.set_ylabel("Fidelity", rotation = 90, fontsize = 12)
ax2.set_xlabel(r"t ($\xi^{-1}$)")
ax2.legend(loc = 'lower right')
ax2.set_title('(b)', loc = "right", fontsize = 10)

ax4.plot(time_span, population1, label = r"$P_1$")
ax4.plot(time_span, population2, label = r"$P_2$")
ax4.plot(time_span, population3, label = r"$P_3$")
ax4.set_ylabel("Populations", rotation = 90, fontsize = 12)
ax4.set_xlabel(r"t ($\xi^{-1}$)")
ax4.legend()
ax4.set_title('(c)', loc = "right", fontsize = 10)